## Igualamos la cantidad de columnas a 25 para cada fila

In [ ]:
import csv

csv.field_size_limit(10000000)
input_file = open('biden_cash_enc.txt', 'r', encoding='utf-8')
# considero que es mejora cambiar la codificación aquí
output_file = open('biden_cash_clean_enc.txt', 'w', encoding='utf-8')

In [ ]:
reader = csv.reader(input_file, delimiter='|')
writer = csv.writer(output_file, delimiter='|')

num_cols = 25  # Número de columnas esperado en cada fila
for row in reader:
    if len(row) < num_cols:
        # Agregar valores faltantes
        row.extend([''] * (num_cols - len(row)))
    elif len(row) > num_cols:
        # Eliminar columnas adicionales
        row = row[:num_cols]
    writer.writerow(row)

input_file.close()
output_file.close()

## Leemos el archivo preprocesado en el dataframe principal (df)

In [ ]:
import pandas as pd

df = pd.read_csv('biden_cash_clean.txt', sep='|', header=None, on_bad_lines='skip', dtype={0: str, 17: str, 18: str, 19: str, 20: str, 21: str, 22: str, 23: str})

In [ ]:
df

### Algunos caracteres no se transcodificaron de forma correcta, se procede a buscarlos

In [ ]:
with open('biden_cash_clean_enc.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    unique_chars = set(content)
    result = ''.join(unique_chars)

In [ ]:
result

In [ ]:
import re
valores = r'[^\w\s\x01-\x20\x01-\x20\uf8ff\\/\()\,\[\]\.\-\–\']'

result = re.findall(valores, result)
result = ''.join(result)

In [ ]:
result

In [ ]:
regex = '[' + re.escape(result) + ']'

#### Tratamos de reemplazar algunos caracteres conocidos por su equivalente

In [ ]:
reemplazo = {
    '√©': 'é', '√°': 'á', '√¥': 'ô', '¬†': ' ',
    '††': ' ', '√¢': 'â', '√£': 'ã', '©ú': 'Ü',
    '©ñ': 'Ö', '©≠': 'í', 'Ё®': 'ó', '&eacute': 'é', '&aacute': 'á',
    '&iacute': 'í', 'ú≠': 'í', '&oacute': 'ó', 'ú°': 'á', '&uacute': 'ú'
}

# columnas = [3]
columnas = [3, 9, 10, 11, 12]

for columna in columnas:
    for c, v in reemplazo.items():
        df[columna] = df[columna].str.replace(c, v)
        # print(columna, c, v)
# df[columnas] = df[columnas].replace(reemplazo, inplace=False)

#### Se obtiene el dataframe (matches) con los caracteres que no se transcodificaron de forma correcta y no se pudieron reemplazar

In [ ]:
matches = df[df[3].str.contains(regex, na=False)]

In [ ]:
matches

## Eliminamos los elementos del dataframe matches del dataframe principal

In [ ]:
diff = pd.merge(df, matches, how='outer', indicator=True)
diff = diff[diff['_merge'] == 'left_only'].drop('_merge', axis=1)

diff

### Renombramos las columnas con el posible nombre del dato que contiene

In [ ]:
diff.rename(columns={
    0: 'numero_tarjeta',
    1: 'vencimiento',
    2: 'ccv',
    3: 'nombre_titular',
    4: 'nombre_banco',
    5: 'tarjeta',
    6: 'nivel_tarjeta',
    7: 'tipo_tarjeta',
    8: 'direction_titular',
    9: 'abreviatura_cp',
    10: 'ciudad_titular',
    11: 'code_postal',
    12: 'pais_titular',
    15: 'correo_titular',
    16: 'numero_titular'
}, inplace=True)

In [ ]:
diff

#### Para tener un mejor conocimiento de los datos, les agregue una descripcion:

* numero_tarjeta: Número de identificación único asignado a la tarjeta de crédito o débito.
* vencimiento: Fecha de expiración de la tarjeta de crédito o débito.
* ccv: Código de seguridad de tres o cuatro dígitos ubicado en la parte trasera de la tarjeta de crédito o débito.
* nombre_titular: Nombre completo del titular de la tarjeta de crédito o débito.
* nombre_banco: Nombre del banco emisor de la tarjeta de crédito o débito.
* tarjeta: Tipo de tarjeta de crédito o débito (por ejemplo, Visa, Mastercard, American Express, etc.).
* nivel_tarjeta: Nivel de la tarjeta de crédito o débito (por ejemplo, clásica, oro, platino, etc.).
* tipo_tarjeta: Tipo de tarjeta de crédito o débito (por ejemplo, personal, empresarial, estudiantil, etc.).
* direction_titular: Dirección del titular de la tarjeta de crédito o débito.
* abreviatura_cp: Abreviatura del código postal del titular de la tarjeta de crédito o débito.
* ciudad_titular: Ciudad de residencia del titular de la tarjeta de crédito o débito.
* code_postal: Código postal del titular de la tarjeta de crédito o débito.
* pais_titular: País de residencia del titular de la tarjeta de crédito o débito.
* correo_titular: Correo electrónico del titular de la tarjeta de crédito o débito.
* numero_titular: Número de teléfono del titular de la tarjeta de crédito o débito.